In [37]:
from pathlib import Path
import pandas as pd
import ast

def parse_listlike(x):
    """Turn strings like '[1, 2, 3]' into lists; leave others as-is."""
    if isinstance(x, str):
        s = x.strip()
        if s.startswith('[') and s.endswith(']'):
            try:
                return ast.literal_eval(s)
            except Exception:
                return x
    return x

def excel_serial_to_datetime(series):
    """Excel serial days (since 1899-12-30) -> pandas datetime64[ns]."""
    ser = pd.to_numeric(series, errors='coerce')
    return pd.to_datetime('1899-12-30') + pd.to_timedelta(ser, unit='D')

def convert_folder(in_dir, out_dir, add_time=None):
    """
    Convert ClockDate/ClockDateTime Excel-serials to real datetimes for all CSVs.

    in_dir  : input folder with CSV files
    out_dir : output folder to write converted CSVs
    add_time: None (default), 's' to add Time as seconds, or 'm' to add minutes
    """
    in_dir  = Path(in_dir)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    for csv_path in sorted(in_dir.glob('*.csv')):
        # robust read with fallback encoding
        try:
            df = pd.read_csv(csv_path)
        except UnicodeDecodeError:
            df = pd.read_csv(csv_path, encoding='latin-1')

        # normalize first two column names (files sometimes have messy headers)
        cols = list(df.columns)
        if len(cols) >= 1: cols[0] = 'ClockDateTime'
        if len(cols) >= 2: cols[1] = 'Time'
        df.columns = cols

        # optional parse of list-like strings
        df = df.applymap(parse_listlike)

        # convert first column:
        # if it's already a proper datetime string, this keeps it;
        # if it's an Excel serial (e.g., 44922.86), this converts correctly.
        cdt = df['ClockDateTime']
        # try serial -> datetime
        converted = excel_serial_to_datetime(cdt)
        # where that failed (NaT), try parsing as regular datetime strings
        fallback = pd.to_datetime(cdt, errors='coerce', infer_datetime_format=True)
        df['ClockDateTime'] = converted.fillna(fallback)

        # optionally add the Time offset
        if add_time in ('s', 'm'):
            # make Time numeric; non-numeric -> NaN -> treated as 0 offset
            tnum = pd.to_numeric(df['Time'], errors='coerce').fillna(0)
            df['ClockDateTime'] = df['ClockDateTime'] + pd.to_timedelta(
                tnum, unit=('s' if add_time == 's' else 'm')
            )

        # write out with same filename
        df.to_csv(out_dir / csv_path.name, index=False)


convert_folder(
     in_dir=r"C:\Users\Haiya\Downloads\OneDrive_2025-09-19\Deidentified-all cohort - Copy",
     out_dir=r"C:\Users\Haiya\Downloads\OneDrive_2025-09-19\patient_data",
     add_time=None  # or 's' / 'm' if Time is seconds/minutes
 )


C:\Users\Haiya\AppData\Local\Temp\ipykernel_1516\1012938455.py:36: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
C:\Users\Haiya\AppData\Local\Temp\ipykernel_1516\1012938455.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(parse_listlike)
C:\Users\Haiya\AppData\Local\Temp\ipykernel_1516\1012938455.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fallback = pd.to_datetime(cdt, errors='coerce', infer_datetime_format=True)
C:\Users\Haiya\AppData\Local\Temp\ipyk